In [1]:
import hdf5_getters

In [2]:
# sample file
h5 = hdf5_getters.open_h5_file_read('MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5')

# in byte format --> decode to string
print("Sample Artist: %s, Song: %s" % \
      (hdf5_getters.get_artist_name(h5).decode('UTF-8'), hdf5_getters.get_title(h5).decode('UTF-8')))

Sample Artist: Casual, Song: I Didn't Mean To


In [3]:
# iterate over files
# modified version of:
# https://labrosa.ee.columbia.edu/millionsong/pages/iterate-over-all-songs

import os
import glob

def count_and_get_files(basedir,ext='.h5') :
    cnt = 0
    all_files = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for file in files:
            all_files.append(file)
        cnt += len(files)
    return cnt, all_files

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .appName("Exploring Million Song Dataset") \
    .getOrCreate()

sc = spark.sparkContext

In [5]:
file_names_rdd = sc.parallelize(count_and_get_files('MillionSongSubset')[1])

In [6]:
def get_artist(filename):
    h5 = hdf5_getters.open_h5_file_read(filename)
    return hdf5_getters.get_artist_name(h5).decode('UTF-8')

artist_names_rdd = file_names_rdd.map(get_artist)

In [7]:
artist_names_rdd.take(5)

['Raphaël',
 'Julie Zenatti',
 'The Baltimore Consort',
 'I Hate Sally',
 'Orlando Pops Orchestra']